# TCGA 

## Initial Setup

Connects to Google Drive
Installs and imports dependancies
Run whether starting or returning

In [1]:
from multiprocessing import Process
import numpy as np
from numpy import asarray
import os
import pandas as pd
import shutil
from sys import platform
import glob
from matplotlib import pyplot as plt
from PIL import Image
from tqdm import tqdm
from matplotlib import pyplot as plt
import random 
from PIL import Image
plt.style.use("ggplot")
import gc
# import sklearn
from sklearn.model_selection import train_test_split
import pickle
import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor




In [2]:
print(os.getcwd())

/home_ssd2/deanak


In [3]:
import os
process_count = len(os.sched_getaffinity(0))
print('Processors available: ', process_count)

Processors available:  32


In [4]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Dec 20 17:12:07 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.86.10              Driver Version: 535.86.10    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2080 Ti     On  | 00000000:43:00.0 Off |                  N/A |
|140%   77C    P2             281W / 300W |   5542MiB / 11264MiB |     93%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Working with tiles

Continue from here if resuming work


In [6]:
%cd CancerPrediction
df_datalabels = pd.read_csv('slides/this_model_datalabels.csv')
manifest_filename = 'slides/gdc_manifest_train.txt'
df_manifest = pd.read_csv(manifest_filename, sep='\t')
clinic_filename = 'CombinedClinicalManifestData.csv'
df_clinic = pd.read_csv(clinic_filename)
df_reduced = pd.read_csv('slides/reduce_model_labels.csv')

label = 'ER_Label'
status = 'brca_shared-er_status_by_ihc'
print(len(df_datalabels))
df_datalabels.head()
# print('e2bf3d1f-c5cd-4551-bf2d-0a4fe79ae2da' in df_datalabels.id.values)

/home_ssd2/deanak/CancerPrediction
3068


,Unnamed: 0,id,filename,brca_shared-pr_status_by_ihc,brca_shared-er_status_by_ihc,brca_shared-her2_status_by_ihc,Path,ER_Label,PR_Label,HER_Label,NNN_label,Tile_DF
0,1.0,b69e1106-0c5b-4fd2-bd00-f3aa3ea9eda8,TCGA-BH-A18V-06A-01-TSA.BEF73FC6-FB7D-47EE-867...,Negative,Negative,Negative,/slides/b69e1106-0c5b-4fd2-bd00-f3aa3ea9eda8/t...,0,0,0,1,slides/b69e1106-0c5b-4fd2-bd00-f3aa3ea9eda8/ti...
1,3.0,fd339777-4def-4466-8fee-9762b7bab9f9,TCGA-C8-A12L-01A-01-BSA.f97cb175-3951-4f67-ac5...,Negative,Negative,Equivocal,/slides/fd339777-4def-4466-8fee-9762b7bab9f9/t...,0,0,2,0,slides/fd339777-4def-4466-8fee-9762b7bab9f9/ti...
2,4.0,ba56d49e-1907-44fd-93aa-34bfc566dd59,TCGA-AN-A0XT-01A-01-TSA.14d9f5ee-14a2-4049-935...,Negative,Positive,Negative,/slides/ba56d49e-1907-44fd-93aa-34bfc566dd59/t...,1,0,0,0,slides/ba56d49e-1907-44fd-93aa-34bfc566dd59/ti...
3,5.0,ca7e0d59-7c28-41ba-8c55-59b69ab80d07,TCGA-A8-A091-01A-01-TS1.c06440ae-88c4-4ed8-a1b...,Negative,Positive,Negative,/slides/ca7e0d59-7c28-41ba-8c55-59b69ab80d07/t...,1,0,0,0,slides/ca7e0d59-7c28-41ba-8c55-59b69ab80d07/ti...
4,10.0,f668d072-2b1e-4d18-a746-3229088fb799,TCGA-A7-A13E-01B-06-BS6.4796C3A1-71ED-4757-B9E...,Negative,Positive,Equivocal,/slides/f668d072-2b1e-4d18-a746-3229088fb799/t...,1,0,2,0,slides/f668d072-2b1e-4d18-a746-3229088fb799/ti...


In [7]:
df_model_datalabels_new = pd.read_csv('slides/model_datalabels_new.csv')

print(len(df_model_datalabels_new))
df_model_datalabels_new.head()


387


,id,filename,brca_shared-pr_status_by_ihc,brca_shared-er_status_by_ihc,brca_shared-her2_status_by_ihc,Path,ER_Label,PR_Label,HER_Label,NNN_label,Tile_DF
0,2ed1ad16-98b3-4941-b223-47a1af8efdae,TCGA-BH-A18N-11A-02-TSB.c1360bc0-7e02-4847-a9d...,Positive,Positive,Negative,/slides/2ed1ad16-98b3-4941-b223-47a1af8efdae/t...,1,1,0,0,slides/2ed1ad16-98b3-4941-b223-47a1af8efdae/ti...
1,5df4c1e0-8ece-4e5f-8deb-667685cf4ca8,TCGA-BH-A18N-11A-04-TSD.dc113c2d-2491-48d2-817...,Positive,Positive,Negative,/slides/5df4c1e0-8ece-4e5f-8deb-667685cf4ca8/t...,1,1,0,0,slides/5df4c1e0-8ece-4e5f-8deb-667685cf4ca8/ti...
2,11a5dc17-fe05-4949-a5f2-be3e0a1069c1,TCGA-BH-A18N-11A-03-TSC.812dc8a3-8b4b-4c72-b9a...,Positive,Positive,Negative,/slides/11a5dc17-fe05-4949-a5f2-be3e0a1069c1/t...,1,1,0,0,slides/11a5dc17-fe05-4949-a5f2-be3e0a1069c1/ti...
3,9f951971-317b-4a02-9998-12d9e05deb1e,TCGA-BH-A18N-11A-01-TSA.67433f2f-b31c-420a-adb...,Positive,Positive,Negative,/slides/9f951971-317b-4a02-9998-12d9e05deb1e/t...,1,1,0,0,slides/9f951971-317b-4a02-9998-12d9e05deb1e/ti...
4,262b8c2d-7c66-403b-827f-e870d6c1155d,TCGA-AC-A2FK-11A-01-TSA.5CBEF273-619D-47FD-844...,Positive,Positive,NaN,/slides/262b8c2d-7c66-403b-827f-e870d6c1155d/t...,1,1,2,0,slides/262b8c2d-7c66-403b-827f-e870d6c1155d/ti...


In [8]:
df_datalabels_new = pd.concat([df_datalabels,df_model_datalabels_new]).drop_duplicates().reset_index(drop=True)
print(len(df_datalabels_new))
df_datalabels_new.head()

3068


,Unnamed: 0,id,filename,brca_shared-pr_status_by_ihc,brca_shared-er_status_by_ihc,brca_shared-her2_status_by_ihc,Path,ER_Label,PR_Label,HER_Label,NNN_label,Tile_DF
0,1.0,b69e1106-0c5b-4fd2-bd00-f3aa3ea9eda8,TCGA-BH-A18V-06A-01-TSA.BEF73FC6-FB7D-47EE-867...,Negative,Negative,Negative,/slides/b69e1106-0c5b-4fd2-bd00-f3aa3ea9eda8/t...,0,0,0,1,slides/b69e1106-0c5b-4fd2-bd00-f3aa3ea9eda8/ti...
1,3.0,fd339777-4def-4466-8fee-9762b7bab9f9,TCGA-C8-A12L-01A-01-BSA.f97cb175-3951-4f67-ac5...,Negative,Negative,Equivocal,/slides/fd339777-4def-4466-8fee-9762b7bab9f9/t...,0,0,2,0,slides/fd339777-4def-4466-8fee-9762b7bab9f9/ti...
2,4.0,ba56d49e-1907-44fd-93aa-34bfc566dd59,TCGA-AN-A0XT-01A-01-TSA.14d9f5ee-14a2-4049-935...,Negative,Positive,Negative,/slides/ba56d49e-1907-44fd-93aa-34bfc566dd59/t...,1,0,0,0,slides/ba56d49e-1907-44fd-93aa-34bfc566dd59/ti...
3,5.0,ca7e0d59-7c28-41ba-8c55-59b69ab80d07,TCGA-A8-A091-01A-01-TS1.c06440ae-88c4-4ed8-a1b...,Negative,Positive,Negative,/slides/ca7e0d59-7c28-41ba-8c55-59b69ab80d07/t...,1,0,0,0,slides/ca7e0d59-7c28-41ba-8c55-59b69ab80d07/ti...
4,10.0,f668d072-2b1e-4d18-a746-3229088fb799,TCGA-A7-A13E-01B-06-BS6.4796C3A1-71ED-4757-B9E...,Negative,Positive,Equivocal,/slides/f668d072-2b1e-4d18-a746-3229088fb799/t...,1,0,2,0,slides/f668d072-2b1e-4d18-a746-3229088fb799/ti...


In [9]:
df_datalabels = df_datalabels_new
print(len(df_datalabels))
df_datalabels.to_csv('slides/this_model_datalabels.csv', index=False)
df_datalabels.head()

3068


,Unnamed: 0,id,filename,brca_shared-pr_status_by_ihc,brca_shared-er_status_by_ihc,brca_shared-her2_status_by_ihc,Path,ER_Label,PR_Label,HER_Label,NNN_label,Tile_DF
0,1.0,b69e1106-0c5b-4fd2-bd00-f3aa3ea9eda8,TCGA-BH-A18V-06A-01-TSA.BEF73FC6-FB7D-47EE-867...,Negative,Negative,Negative,/slides/b69e1106-0c5b-4fd2-bd00-f3aa3ea9eda8/t...,0,0,0,1,slides/b69e1106-0c5b-4fd2-bd00-f3aa3ea9eda8/ti...
1,3.0,fd339777-4def-4466-8fee-9762b7bab9f9,TCGA-C8-A12L-01A-01-BSA.f97cb175-3951-4f67-ac5...,Negative,Negative,Equivocal,/slides/fd339777-4def-4466-8fee-9762b7bab9f9/t...,0,0,2,0,slides/fd339777-4def-4466-8fee-9762b7bab9f9/ti...
2,4.0,ba56d49e-1907-44fd-93aa-34bfc566dd59,TCGA-AN-A0XT-01A-01-TSA.14d9f5ee-14a2-4049-935...,Negative,Positive,Negative,/slides/ba56d49e-1907-44fd-93aa-34bfc566dd59/t...,1,0,0,0,slides/ba56d49e-1907-44fd-93aa-34bfc566dd59/ti...
3,5.0,ca7e0d59-7c28-41ba-8c55-59b69ab80d07,TCGA-A8-A091-01A-01-TS1.c06440ae-88c4-4ed8-a1b...,Negative,Positive,Negative,/slides/ca7e0d59-7c28-41ba-8c55-59b69ab80d07/t...,1,0,0,0,slides/ca7e0d59-7c28-41ba-8c55-59b69ab80d07/ti...
4,10.0,f668d072-2b1e-4d18-a746-3229088fb799,TCGA-A7-A13E-01B-06-BS6.4796C3A1-71ED-4757-B9E...,Negative,Positive,Equivocal,/slides/f668d072-2b1e-4d18-a746-3229088fb799/t...,1,0,2,0,slides/f668d072-2b1e-4d18-a746-3229088fb799/ti...


In [10]:
# Drops images with insufficient number of tiles
n = 50 # Minimum number of tiles to keep WSIs
drop = []

for index, row in df_datalabels.iterrows():
  tile_count = len(pd.read_csv(row.Tile_DF))
  if tile_count < n:
    # print(f'{row.id}: {tile_count}')
    drop.append(row.id)
print(f'Number of dropped images: {len(drop)}')  
df_reduced = df_datalabels[~df_datalabels.id.isin(drop)]
df_reduced.to_csv('slides/reduce_model_labels.csv', index=False)
del tile_count, drop
print(len(df_reduced))
df_reduced.head()

Number of dropped images: 243
2825


,Unnamed: 0,id,filename,brca_shared-pr_status_by_ihc,brca_shared-er_status_by_ihc,brca_shared-her2_status_by_ihc,Path,ER_Label,PR_Label,HER_Label,NNN_label,Tile_DF
0,1.0,b69e1106-0c5b-4fd2-bd00-f3aa3ea9eda8,TCGA-BH-A18V-06A-01-TSA.BEF73FC6-FB7D-47EE-867...,Negative,Negative,Negative,/slides/b69e1106-0c5b-4fd2-bd00-f3aa3ea9eda8/t...,0,0,0,1,slides/b69e1106-0c5b-4fd2-bd00-f3aa3ea9eda8/ti...
1,3.0,fd339777-4def-4466-8fee-9762b7bab9f9,TCGA-C8-A12L-01A-01-BSA.f97cb175-3951-4f67-ac5...,Negative,Negative,Equivocal,/slides/fd339777-4def-4466-8fee-9762b7bab9f9/t...,0,0,2,0,slides/fd339777-4def-4466-8fee-9762b7bab9f9/ti...
2,4.0,ba56d49e-1907-44fd-93aa-34bfc566dd59,TCGA-AN-A0XT-01A-01-TSA.14d9f5ee-14a2-4049-935...,Negative,Positive,Negative,/slides/ba56d49e-1907-44fd-93aa-34bfc566dd59/t...,1,0,0,0,slides/ba56d49e-1907-44fd-93aa-34bfc566dd59/ti...
3,5.0,ca7e0d59-7c28-41ba-8c55-59b69ab80d07,TCGA-A8-A091-01A-01-TS1.c06440ae-88c4-4ed8-a1b...,Negative,Positive,Negative,/slides/ca7e0d59-7c28-41ba-8c55-59b69ab80d07/t...,1,0,0,0,slides/ca7e0d59-7c28-41ba-8c55-59b69ab80d07/ti...
4,10.0,f668d072-2b1e-4d18-a746-3229088fb799,TCGA-A7-A13E-01B-06-BS6.4796C3A1-71ED-4757-B9E...,Negative,Positive,Equivocal,/slides/f668d072-2b1e-4d18-a746-3229088fb799/t...,1,0,2,0,slides/f668d072-2b1e-4d18-a746-3229088fb799/ti...


In [10]:
########### REPOPULATE WSI TILE DATAFRAMES ############

path = 'slides'
name = 'tile_data.csv'
df_datalabels.reset_index(drop=True)
for img_dir in os.listdir(path): 
  if os.path.isdir(os.path.join(path, img_dir)) and 'ipynb' not in img_dir:
    file_path = []
    for file in glob.glob(os.path.join(path, img_dir, 'tiles', '*.png')):
      file_path.append(file)
    df = pd.DataFrame()
    df['TilePath'] = file_path
    df.to_csv(os.path.join(path, img_dir, name), index=False)

df_datalabels['Tile_DF'] = df_datalabels.apply(lambda x: os.path.join(path, x.id, name), axis=1)
df_datalabels = df_datalabels.loc[:, ~df_datalabels.columns.str.contains('index')]
df_datalabels = df_datalabels.loc[:, ~df_datalabels.columns.str.contains('Unnamed')]
df_datalabels.to_csv('slides/model_datalabels.csv', index=False)

# Memory cleanup
del file_path, df
print(len(df_datalabels))
df_datalabels.head()

3068


,id,filename,brca_shared-pr_status_by_ihc,brca_shared-er_status_by_ihc,brca_shared-her2_status_by_ihc,Path,ER_Label,PR_Label,HER_Label,NNN_label,Tile_DF
0,b69e1106-0c5b-4fd2-bd00-f3aa3ea9eda8,TCGA-BH-A18V-06A-01-TSA.BEF73FC6-FB7D-47EE-867...,Negative,Negative,Negative,/slides/b69e1106-0c5b-4fd2-bd00-f3aa3ea9eda8/t...,0,0,0,1,slides/b69e1106-0c5b-4fd2-bd00-f3aa3ea9eda8/ti...
1,fd339777-4def-4466-8fee-9762b7bab9f9,TCGA-C8-A12L-01A-01-BSA.f97cb175-3951-4f67-ac5...,Negative,Negative,Equivocal,/slides/fd339777-4def-4466-8fee-9762b7bab9f9/t...,0,0,2,0,slides/fd339777-4def-4466-8fee-9762b7bab9f9/ti...
2,ba56d49e-1907-44fd-93aa-34bfc566dd59,TCGA-AN-A0XT-01A-01-TSA.14d9f5ee-14a2-4049-935...,Negative,Positive,Negative,/slides/ba56d49e-1907-44fd-93aa-34bfc566dd59/t...,1,0,0,0,slides/ba56d49e-1907-44fd-93aa-34bfc566dd59/ti...
3,ca7e0d59-7c28-41ba-8c55-59b69ab80d07,TCGA-A8-A091-01A-01-TS1.c06440ae-88c4-4ed8-a1b...,Negative,Positive,Negative,/slides/ca7e0d59-7c28-41ba-8c55-59b69ab80d07/t...,1,0,0,0,slides/ca7e0d59-7c28-41ba-8c55-59b69ab80d07/ti...
4,f668d072-2b1e-4d18-a746-3229088fb799,TCGA-A7-A13E-01B-06-BS6.4796C3A1-71ED-4757-B9E...,Negative,Positive,Equivocal,/slides/f668d072-2b1e-4d18-a746-3229088fb799/t...,1,0,2,0,slides/f668d072-2b1e-4d18-a746-3229088fb799/ti...
